In [ ]:
! pip install langchain-anthropic langgraph

In [ ]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("ANTHROPIC_API_KEY")

## Few Shot Examples

You can think of this as "episodic" memory. 

These are the examples that the LLM can use to help it extract memories correctly.

Let's build on what we did in the previous lesson.

First, we define a general schema for any memory.

The schema has a `memory_type` and a `memory_content`.

![](./img/memory_course_episodic.png)



In [37]:
from pydantic import BaseModel, Field
from typing import List
from langchain_anthropic import ChatAnthropic

# Schema for structured output
class Memory(BaseModel):
    memory_type: str = Field(None, description="Type of memory to extract.")
    memory_content: str = Field(None, description="Specific content of the memory.")

class Memories(BaseModel):
    memories: List[Memory] = Field(None, description="List of memories to extract.")

# Define and augment LLM with structured output
llm = ChatAnthropic(model="claude-3-5-sonnet-latest")
structured_llm = llm.with_structured_output(Memories)

Now let's load our instructions for memory extraction, as we did before.

But, we'll also load some examples that explain how to extract memories correctly.

In [38]:
import sys
sys.path.append('..')
import src.memory_course.prompts
import src.memory_course.examples
import src.memory_course.utils

# Reload the module in case any changes were made
import importlib
importlib.reload(src.memory_course.prompts)
importlib.reload(src.memory_course.examples)
importlib.reload(src.memory_course.utils)

# Then import the instructions and examples
from src.memory_course.utils import format_few_shot_examples
from src.memory_course.examples import example_input, example_output
from src.memory_course.prompts import memory_collection_extraction_instructions, update_few_shot_examples_instructions, collection_extraction_input

As before, we'll use the store to store our instructions and examples.

In [39]:
# Save instructions to the store
from datetime import datetime
from langgraph.store.memory import InMemoryStore

in_memory_store = InMemoryStore()
namespace = ("journal","instructions")
key = "instructions_extraction"
in_memory_store.put(namespace, key, {"instructions": memory_collection_extraction_instructions})

namespace = ("journal","examples")
key = f"example_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
in_memory_store.put(namespace, key, {"input": example_input, "output": example_output})

We can now fetch examples from the store.

In [40]:
from src.memory_course.utils import format_few_shot_examples
few_shot_examples = in_memory_store.search(("journal", "examples"))

# Format the examples for the LLM
episodic_memory_formatted = format_few_shot_examples(few_shot_examples)
episodic_memory_formatted

'Input:\n\nToday was a mix of ups and downs. Started the morning feeling energized after my new morning routine, but got a bit overwhelmed with the project deadlines in the afternoon. \n\nNeed to remember to:\n- Email Sarah about the client presentation by Thursday\n- Schedule dentist appointment\n- Pick up groceries for weekend dinner party\n\nHad an interesting thought during my walk - what if we incorporated AI into our customer feedback system? Could potentially automate the initial response and categorization. Would need to research existing solutions first.\n\nDespite the stress, I\'m optimistic about where things are heading. The team\'s really coming together on the new initiative.\n\nOutput:\n\n{\n    "memories": [\n        {\n            "memory_type": "SENTIMENT",\n            "memory_content": "Mixed emotions with overall optimistic tone. Started energized, experienced afternoon overwhelm with deadlines, but ended positively about team progress. Key phrases: \'energized\', 

As before, we define a function to extract memories from a journal entry that uses both instruction and few shot examples.

In [41]:
from langchain_core.messages import HumanMessage, SystemMessage

def extract_memories(journal_entry: str) -> Memories:
    """Extract memories from a journal entry.
    
    This function analyzes a journal entry and extracts memories
    
    Args:
        journal_entry (str): The text of the journal entry to analyze
        
    Returns:
        Memories (list): A structured list of Memory objects"""

    # Get the instructions from the store
    namespace = ("journal","instructions")
    key = "instructions_extraction"
    instructions = in_memory_store.get(namespace, key)

    # Get the examples from the store
    few_shot_examples = in_memory_store.search(("journal", "examples"))
    few_shot_examples_formatted = format_few_shot_examples(few_shot_examples)

    # Format the instructions
    memory_extraction_instructions_formatted = collection_extraction_input.format(
        procedural_memory_instructions=instructions.value['instructions'],
        few_shot_examples_formatted=few_shot_examples_formatted
    )

    # Extract memories
    memories = structured_llm.invoke([SystemMessage(content=memory_extraction_instructions_formatted),
                                      HumanMessage(content=f"Extract memory from <context> {journal_entry} </context>")])
    
    return memories

Now, we can test extraction from a fake journal entry.

In [9]:
# Test extraction of memories
journal_entry = """
Really productive coding session this morning! The new refactoring approach is working better than expected, though I hit a few snags with the database migrations.

Important tasks for this week:
- Set up meeting with DevOps team about deployment strategy
- Fix unit tests for the payment module by Friday
- Order new laptop charger
- Update documentation for API changes

During the team standup, had a fascinating idea about improving our testing workflow. What if we implemented automatic test generation using LLMs? Could save hours of manual test writing. Would need to evaluate cost and accuracy first.

Even though there's a lot on my plate, feeling pretty confident about the sprint goals. The recent architecture changes are already showing positive results."""

# Run extraction
memories = extract_memories(journal_entry)

# Review
for memory in memories.memories:
    print("=== Extracted Memory ===")
    print(f"\n{memory.memory_type.upper()}:")
    print(f"- {memory.memory_content}\n")

=== Extracted Memory ===

SENTIMENT:
- Overall positive and confident mood. Productive coding session and optimistic about sprint goals. Some minor challenges with database migrations but maintaining positive outlook. Key phrases: 'working better than expected', 'feeling pretty confident', 'showing positive results'

=== Extracted Memory ===

TODO:
- Set up meeting with DevOps team about deployment strategy

=== Extracted Memory ===

TODO:
- Fix unit tests for payment module - Deadline: Friday

=== Extracted Memory ===

TODO:
- Order new laptop charger

=== Extracted Memory ===

TODO:
- Update documentation for API changes

=== Extracted Memory ===

IDEA:
- Implement automatic test generation using LLMs to reduce manual test writing time. Requirements: Need to evaluate cost and accuracy implications



In a journal, we want the ability to update examples directly based upon user feedback. 

Let's look at the current examples.


In [28]:
# Get the examples from the store
few_shot_examples = in_memory_store.search(("journal", "examples"))
few_shot_examples_formatted = format_few_shot_examples(few_shot_examples)
few_shot_examples_formatted

'Input:\n\nToday was a mix of ups and downs. Started the morning feeling energized after my new morning routine, but got a bit overwhelmed with the project deadlines in the afternoon. \n\nNeed to remember to:\n- Email Sarah about the client presentation by Thursday\n- Schedule dentist appointment\n- Pick up groceries for weekend dinner party\n\nHad an interesting thought during my walk - what if we incorporated AI into our customer feedback system? Could potentially automate the initial response and categorization. Would need to research existing solutions first.\n\nDespite the stress, I\'m optimistic about where things are heading. The team\'s really coming together on the new initiative.\n\nOutput:\n\n{\n    "memories": [\n        {\n            "memory_type": "SENTIMENT",\n            "memory_content": "Mixed emotions with overall optimistic tone. Started energized, experienced afternoon overwhelm with deadlines, but ended positively about team progress. Key phrases: \'energized\', 

In [32]:
# Schema for a example
class Example(BaseModel):
    example_input: str = Field(None, description="Input for the example.")
    example_output: str = Field(None, description="Output for the example.")

# List of examples 
class Examples(BaseModel):
    examples: List[Example] = Field(None, description="List examples.")

In [33]:
# Function for updating the instructions
def update_few_shot_examples(user_feedback: str) -> str:
    """Few shot examples can be updated based on user feedback.
        
    Args:
        user_feedback (str): User feedback about the memory extraction process
        
    Returns:
        None"""

    # Get the examples from the store
    namespace = ("journal","examples")
    few_shot_examples = in_memory_store.search(namespace)
    few_shot_examples_formatted = format_few_shot_examples(few_shot_examples)

    # Create new few shot examples from the user feedback
    structured_llm = llm.with_structured_output(Examples)
    updated_few_shot_examples = structured_llm.invoke([SystemMessage(content=update_few_shot_examples_instructions.format(current_examples=few_shot_examples_formatted)),
                                            HumanMessage(content=f"<Feedback> {user_feedback} </Feedback>")])
    
    # Add the new few shot examples to the store
    for example in updated_few_shot_examples.examples:
        key = f"example_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        in_memory_store.put(namespace, key, {"input": example.example_input, "output": example.example_output})

update_few_shot_examples("If I talk about something 'interesting' related to work, then classify it and save it as an idea.")


In [35]:
# Get the examples from the store
few_shot_examples = in_memory_store.search(("journal", "examples"))
few_shot_examples[-1]

Item(namespace=['journal', 'examples'], key='example_20250203_133145', value={'input': 'Interesting insight from the sales meeting - our competitors are all moving towards subscription models.', 'output': '{"memories": [{"memory_type": "IDEA", "memory_content": "Consider subscription model strategy in response to competitor market trends."}]}'}, created_at='2025-02-03T21:31:45.669267+00:00', updated_at='2025-02-03T21:31:45.669268+00:00', score=None)

So now we can:

1. Extract memories from a journal entry given a set of instructions (system message)
2. Update those instructions based upon user feedback directly 
3. Add few shot examples to improve the quality of the extraction
4. Update those few shot examples based upon user feedback directly 

This is a simple example of using and editing both "procedural memory" and "episodic memory". 

In the next lesson, we will look at how how to save the extracted memories and retrieve them later. 